In [ ]:
import string
import re

class Tokenizer():
  """단어 토큰화를 통해 어휘 사전을 구축하고 정수 인덱스값을 반환합니다.

  Attributes
  ----------
  word_dict : dict
      코퍼스에서 학습한 각각의 토큰과 정수 인덱스를 매핑한 어휘 사전입니다.
  
  fit_checker : bool
      어휘 사전(`self.word_dict`)이 구축된 상태일 때 True를 반환합니다.

  Examples
  --------
  >>> import Tokenizer
  >>> sequences = [
  ...              "I like this movie, it's funny.",
  ...              "I hate this movie.", 
  ...              "This was awesome! I like it.",
  ...              "Nice one. I love it."
                  ]
  >>> tokenizer = Tokenizer()
  >>> X = tokenizer.fit_transform(sequences)
  >>> print(tokenizer.word_dict)
  ...   {'oov': 0, 'i': 1, 'like': 2, 'this': 3, 'movie': 4, 'its': 5, 'funny': 6, 'hate': 7, 'was': 8, 'awesome': 9, 'it': 10, 'nice': 11, 'one': 12, 'love': 13}
  >>> print([[round(num,2) for num in lst] for lst in X])
  ...   [[1, 2, 3, 4, 5, 6], [1, 7, 3, 4], [3, 8, 9, 1, 2, 10], [11, 12, 1, 13, 10]]
  """
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences): 
    """텍스트 전처리를 하는 함수입니다.
    
    이 함수는 영어 문장으로 이루어진 코퍼스를 단어 토큰으로 변환합니다.
    입력된 문장에 대해서 소문자로의 변환과 구두점 및 특수문자 제거를 수행합니다. 
    토큰화는 white space 단위로 수행합니다. (문자 공백, 탭, 줄바꿈, 리턴, 폼피드, 세로 탭을 포함)

    Notes
    -----
    이 함수는 의미있는 구두점과 특수문자는 제거하지 않고 유지하는 기능을 지원하지 않습니다.
    사전 과제의 요구사항은 아니지만, 차후 보완 시에는 Penn Treebank Tokenization에 따라
    (1)하이푼으로 구성된 단어는 하나로 유지하고 
    (2)doesn't와 같이 아포스트로피로 '접어'가 함께하는 단어는 분리하는 기능을 추가할 예정입니다.

    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    result : nested list
        각 문장을 토큰화한 결과로, nested list 형태입니다.
    """
    punc = string.punctuation 
    result = [re.sub(r'[{}]'.format(punc),'',doc).lower().split() for doc in sequences]

    return result 
     
  def fit(self, sequences):
    """어휘 사전을 구축하는 함수입니다. 

    `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행한 뒤, 
    각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.

    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    이 함수는 `self.fit_checker=True`를 설정하는 것 외에 별도의 반환값이 없습니다. 
    """
    self.fit_checker = False
    sequences = self.preprocessing(sequences)
    idx = 1
    for doc in sequences:
      for token in doc:
        if token not in list(self.word_dict.keys()):
          self.word_dict.update({token:idx})
          idx += 1

    self.fit_checker = True
  
  def transform(self, sequences):
    """어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다. (`self.fit_checker=False`일 때만 실행)

    `fit`(또는 `fit_transform`) 함수로 학습된 어휘 사전(`self.word_dict`)을 사용합니다.
    어휘 사전에 없는 단어는 'oov'의 index로 변환합니다.

    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    result : nested list
        각 문장의 정수 인덱싱으로, nested list 형태입니다.
    """
    sequences = self.preprocessing(sequences)
    keys = self.word_dict.keys()
    result = []
    if self.fit_checker:
      for doc in sequences:
        idx = []
        for token in doc:
          if token in keys:
            idx.append(self.word_dict.get(token))
          else:
            idx.append(self.word_dict.get('oov'))
        result.append(idx) 
      return result
    
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    """입력 문장을 정수 인덱싱하는 함수입니다.
    
    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    result : nested list
        각 문장의 정수 인덱싱으로, nested list 형태입니다.
    """
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [ ]:
import numpy as np

class TfidfVectorizer:
  """학습한 어휘 사전을 통해 TF-IDF를 계산해 리스트로 반환합니다.
  
  TF-IDF는 특정 단어가 문서 집합 내에서 얼마나 중요한 지를 나타내는 통계적 수치입니다.
  TF(단어 빈도)와 IDF(역문서 빈도)의 곱으로 얻을 수 있습니다.
    - TF(단어 빈도)는 학습한 문서에 각각의 단어가 나타난 횟수를 뜻합니다.
    - IDF(역문서빈도)는 전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 것을 의미합니다.

  이에 관해 더 상세한 설명이 필요하다면 개별 함수의 해설란을 참조해주세요.
  
  Attributes
  ----------
  tokenizer : object
      tokenizer를 불러옵니다.
  fit_checker : bool
      idf(`self.idf`)가 구해진 상태일 때 True를 반환합니다.
  idf : list
      idf 값을 저장한 list입니다.
  tfidf_matrix : list
      TF-IDF 행렬을 변환한 list입니다.

  Examples
  --------
  >>> import Tokenizer
  >>> import TfidfVectorizer
  >>> sequences = [
  ...              "I like this movie, it's funny.",
  ...              "I hate this movie.", 
  ...              "This was awesome! I like it.",
  ...              "Nice one. I love it."
                  ]
  >>> tokenizer = Tokenizer()
  >>> tfidfvectorizer = TfidfVectorizer(tokenizer)
  >>> X = tfidfvectorizer.fit_transform(sequences)
  >>> print(tfidfvectorizer.idf)
  ...   [-0.2231435513142097, 0.28768207245178085, 0.0, ..., 0.6931471805599453, 0.6931471805599453, 0.6931471805599453]
  >>> print(X)
  ...   [[-0.22, 0.29, 0.0, ..., nan, nan, nan], ..., [-0.22, nan, nan, nan, ..., 0.69, 0.69, 0.69]]
  """

  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
    self.idf = []
    self.tfidf_matrix = []
  
  def fit(self, sequences):
    """입력 문장들을 이용해 IDF 값을 구하는 함수입니다.

    IDF(역문서빈도)는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값입니다. 
    전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있습니다.

    이 함수에서 IDF 값은 다음의 공식을 통해 계산됩니다.
      idf(d,t) = log_e[ n / (1 + df(d,t)) ]
      - df(d,t) : 단어 t가 포함된 문장 d의 개수
      - n : 입력된 전체 문장 개수

    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    이 함수는 `self.idf`를 저장하고 `self.fit_checker`를 활성화하는 것 외에 별도의 반환값이 없습니다. 
    """
    tokenized = self.tokenizer.fit_transform(sequences)  
    n = len(sequences)
    df = {}
    for d in tokenized:   
      for t in set(d):    
        if t in df.keys():
          df[t] += 1
        else:
          df[t] = 1

    self.idf = [np.log(n/(1+val)) for val in df.values()]

    self.fit_checker = True

  def transform(self, sequences):
    """입력 문장들을 이용해 TF-IDF 값을 구하는 함수입니다.

    TF-IDF는 어떤 단어가 특정 문서 집합 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치입니다.
    TF(단어 빈도)와 IDF(역문서 빈도)의 곱으로 얻을 수 있습니다.

    이 함수에서 TF-IDF 값은 다음의 공식을 통해 계산됩니다.
      tf-idf(d,t) = tf(d,t) * idf(d,t)
      - tf(d, t) : 문장 d에 단어 t가 나타난 횟수

    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    self.tfidf_matrix : list
        Tf-idf 행렬을 변환한 list입니다. 
    """
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      n_samples = len(sequences)
      n_features = max([max(d) for d in tokenized])
      self.tfidf_matrix = np.full((n_samples,n_features),np.nan)
      # 단어 토큰이 해당 문장에 없을 경우 nan 값을 갖습니다.

      tf = [[k.count(t) for t in k] for k in tokenized]

      for idx_d, d in enumerate(tokenized):
        for idx_t, t in enumerate(d):
          self.tfidf_matrix[idx_d,t-1] = tf[idx_d][idx_t]*self.idf[t-1]

      self.tfidf_matrix = self.tfidf_matrix.tolist()
      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    """입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.
    
    Parameters
    ----------
    sequences : list
        여러 영어 문장이 포함된 list입니다.
          
    Returns
    -------
    self.tfidf_matrix : list
        Tf-idf 행렬을 변환한 list입니다. 
    """
    self.fit(sequences)
    return self.transform(sequences)